# Normalization of Adjacent TMA sections

**Question:** What combination of transformation, scaling and batch regression produces the best batch correction on adjacent tissue sections stained with slighly different panel orders, but same antibody conjugates? 

**Samples:** 
- TMA: constructed from FFPE breast cancer cell lines and normal tonsil and breast tissue.
- Sections: JE-TMA-41, JE-TMA-43, JE-TMA-62
- Scenes: (i.e. TMA cores) 
  - 01, 07: Tonsil 
  - 04: Normal breast
  - 02: HCC1143 (triple negative (TN), basal)
  - 03: HCC3153 (TN, basal)
  - 05, 06: T47D (luminal A, ER+, PR+)
  - 08, 09: BT474 (luminal B, ER+, HER2+)
  - 10, 11: AU565 (HER2+)
  - 12, 13: MDAMB-436 (TN, claudin low)
- Staining: The same fluorphore conjugated antibodies were applied, but not in the same order, and not the same lot.

**Method**: We tested permutations of data transformation, scaling and batch regression. (see table) We evaluated effectiveness using the [kBET metric](https://github.com/theislab/kBET) to test for mixing of batches and the known subtype of the cell lines to ensure separation of biologically relevant cell types. 

| Transformation | Scaling     | Regression  |
|----------------|-------------|-------------|
| raw            | standard    | regress_out |
| log2           | min-max     | combat      |
| arcsinh        | max-abs     |             |
|                | robust      |             |
|                | quantile    |             |
|                | power       |             |
|                | RESTORE     |             |
|                | bg-quantile |             |

In [ ]:
#load libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import copy
import seaborn as sns
import importlib
import scanpy as sc
from sklearn.cluster import KMeans
from sklearn.preprocessing import scale, minmax_scale, StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib as mpl
mpl.rc('figure', max_open_warning = 0)
#os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cmIF')
from mplex_image import visualize as viz, process, preprocess
np.random.seed(1202)

In [ ]:
#go to data location
#change to correct directory
#os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data/cycIF_ValidationStudies/cycIF_Validation/Data')
rootdir = os.getcwd()
s_date = '20201228'
if not os.path.exists(s_date):
    os.mkdir(s_date)
datadir = f'{rootdir}/validation_data'
filterdir = f'{rootdir}/filtered_data'

# Table of contents <a name="contents"></a>
0. [Load Raw Data](#loadold)
1. [Load Filtered Data](#load)
2. [Scaling Function](#func)
3. [Data Transformation](#run)
4. [Scaling](#scale)
   [Visualize](#norm)
5. [kBET Results](#kbet)
6. [Apply Batch Correction](#viznorm)
8. [Cluster Composition](#cluster)
9. [Combat fit/transform](#combat)
9. [Combat fit/transform Application](#combatviz)


# Load Raw Data  <a name="loadold"></a>

[contents](#contents)

In [ ]:
os.chdir(datadir)
df_file = pd.DataFrame(index=os.listdir())
df_file = df_file[df_file.index.str.contains('Filtered')]
df_file['tissue'] = [item.split('_')[1] for item in df_file.index]
df_file['dapi'] = ['DAPI' + item.split('y_DAPI')[1].split('.')[0] for item in df_file.index]

In [ ]:
ls_sample = df_file.tissue.tolist()
d_dapi = dict(zip(df_file.tissue.tolist(),df_file.dapi.tolist()))
df_mi = pd.DataFrame()
df_xy = pd.DataFrame()
df_edge = pd.DataFrame()

for s_sample in ls_sample:
    print(f'loading {s_sample}')
    df_mi = df_mi.append(pd.read_csv(f'{datadir}/features_{s_sample}_FilteredMeanIntensity_{d_dapi[s_sample]}.csv', index_col=0))
    df_xy = df_xy.append(pd.read_csv(f'{datadir}/features_{s_sample}_CentroidXY.csv',index_col=0))
    if os.path.exists(f'{datadir}/features_{s_sample}_EdgeCells153pixels_CentroidXY.csv'):
        df_edge = df_edge.append(pd.read_csv(f'{datadir}/features_{s_sample}_EdgeCells153pixels_CentroidXY.csv',index_col=0))
#scene naming
df_mi.replace({'JE-TMA-43_scene13':'JE-TMA-43_scene12','JE-TMA-43_scene14':'JE-TMA-43_scene13'},inplace=True)
df_mi['cell'] = [item.split('cell')[1] for item in df_mi.index]
df_mi.index = df_mi.slide_scene + '_cell' + df_mi.cell 

df_xy.replace({'JE-TMA-43_scene13':'JE-TMA-43_scene12','JE-TMA-43_scene14':'JE-TMA-43_scene13'},inplace=True)
df_xy['cell'] = [item.split('cell')[1] for item in df_xy.index]
df_xy.index = df_xy.slide_scene + '_cell' + df_xy.cell 

In [ ]:
#select the right CD20 and CK5 fluorophore from JE-TMA-41
ls_index = df_mi[df_mi.index.str.contains('JE-TMA-41') | df_mi.index.str.contains('SMT101Bx2-3')].index
df_mi.loc[ls_index,'CD20_perinuc5'] = df_mi.loc[ls_index,'CD20P_perinuc5']
df_mi.loc[ls_index,'CK5_cytoplasm'] = df_mi.loc[ls_index,'CK5P_cytoplasm']

In [ ]:
ls_drop = ['AR_nuclei','HER2_cytoplasm','ER_nuclei25','CD44_nucadj2','Vim_perinuc5',
           'DAPI1_nuclei','DAPI3_nuclei', 'DAPI4_nuclei',
       'DAPI5_nuclei', 'DAPI6_nuclei', 'DAPI7_nuclei', 'DAPI8_nuclei',
       'CSF1R_perinuc5', 'CAV1_perinuc5', 'BCL2_perinuc5', 'CoxIV_perinuc5',
       'R0c2_perinuc5', 'R0c3_perinuc5', 'R0c4_perinuc5', 'R0c5_perinuc5',
       'R5Qc2_perinuc5', 'R5Qc3_perinuc5', 'R5Qc4_perinuc5', 'R5Qc5_perinuc5',
       'pS6RP_perinuc5', 'CK8_cytoplasm', 'EGFR_cytoplasm', 'MUC1_cytoplasm',
       'DAPI0_nuclei', 'DAPI10_nuclei', 'DAPI5Q_nuclei', 'DAPI9_nuclei',
       'FoxP3_nuclei', 'GRNZB_nuclei', 'H3K4_nuclei', 'PgR_nuclei',
       'R0c2_nuclei', 'R0c3_nuclei', 'R0c4_nuclei', 'R0c5_nuclei',
       'R5Qc2_nuclei', 'R5Qc3_nuclei', 'R5Qc4_nuclei', 'R5Qc5_nuclei',
       'RAD51_nuclei', 'gH2AX_nuclei', 'pRB_nuclei', 'CD20P_perinuc5',
       'CD8R_perinuc5', 'CD4R_perinuc5', 'ColI_perinuc5', 'ColIV_perinuc5',
       'R8Qc2_perinuc5', 'R8Qc3_perinuc5', 'R8Qc4_perinuc5', 'R8Qc5_perinuc5',
       'CK5P_cytoplasm', 'DAPI8Q_nuclei', 'LamB1_nuclei', 'R8Qc2_nuclei',
       'R8Qc3_nuclei', 'R8Qc4_nuclei', 'R8Qc5_nuclei', 'BMP2_perinuc5',
       'Glut1_perinuc5', 'PDGFRa_perinuc5', 'R1c2_perinuc5', 'pAKT_perinuc5',
       'DAPI11_nuclei', 'DAPI12_nuclei', 'H3K27_nuclei', 'HIF1a_nuclei',
       'LamB2_nuclei', 'R1c2_nuclei', 'cPARP_nuclei', 'pERK_nuclei']
#drop markers not shared in all 3 TMAs
df_filter_mi = df_mi.loc[:,~df_mi.columns.isin(ls_drop)]

In [ ]:
# filter out edge cores, edge cells in biopsies
d_filter = {'JE-TMA-41_scene01':(df_xy.DAPI_Y > 5000),'JE-TMA-41_scene03':(df_xy.DAPI_Y > 5000),
            'JE-TMA-41_scene04':(df_xy.DAPI_Y < 1500),'JE-TMA-41_scene05':(df_xy.DAPI_Y > 5000),
            'JE-TMA-41_scene06':(df_xy.DAPI_Y < 1500),'JE-TMA-41_scene08':(df_xy.DAPI_Y < 1500),
            'JE-TMA-41_scene09':(df_xy.DAPI_Y > 5000),'JE-TMA-41_scene11':(df_xy.DAPI_Y < 1500),
            'JE-TMA-43_scene09':(df_xy.DAPI_Y < 1200),'JE-TMA-43_scene13':(df_xy.DAPI_Y < 1500),
            #'JE-TMA-60_scene02':(df_xy.DAPI_Y > 5000),'JE-TMA-60_scene06':(df_xy.DAPI_Y < 1500),
            #'JE-TMA-60_scene08':(df_xy.DAPI_Y > 5000),'JE-TMA-60_scene14':(df_xy.DAPI_X < 1500),
            'JE-TMA-62_scene01':(df_xy.DAPI_Y > 5000),
            'JE-TMA-62_scene02':(df_xy.DAPI_X > 5000),'JE-TMA-62_scene03':(df_xy.DAPI_X < 1000),
            'JE-TMA-62_scene04':(df_xy.DAPI_Y < 1500),'JE-TMA-62_scene06':(df_xy.DAPI_X < 1000),
            'JE-TMA-62_scene08':(df_xy.DAPI_Y > 5000),'JE-TMA-62_scene10':(df_xy.DAPI_Y < 1500),
            'SMT101Bx2-3_scene002':(df_xy.DAPI_Y > 6000),'SMT101Bx2-5_scene002':(df_xy.DAPI_Y > 6000)}
ls_filter_all = []
for s_scene, filtercon in d_filter.items():
    ls_filter = df_xy[(df_xy.slide_scene==s_scene) & filtercon].index.tolist()
    ls_filter_all = ls_filter_all + ls_filter
ls_filter = df_xy[(df_xy.slide_scene=='JE-TMA-60_scene02') &(df_xy.DAPI_X < 1500)].index.tolist()
ls_filter_all = ls_filter_all + ls_filter
#filter edge
ls_filter_all = ls_filter_all + df_edge.index.tolist()
df_filter_mi=df_filter_mi[~df_filter_mi.index.isin(ls_filter_all)]

In [ ]:
%matplotlib inline
# check cells visually
df_cluster = df_filter_mi.loc[:,['HER2_cellmem25','slide_scene']]
df_cluster['cluster'] = 1
df_cluster.drop('HER2_cellmem25',axis=1,inplace=True)

viz.plot_clusters(df_cluster,df_xy)

In [ ]:
df_filter_mi = df_filter_mi.loc[~df_filter_mi.index.isin(df_edge.index)]
df_filter_mi.to_csv('20201228_SMT101-Bx2-Bx2-Bx4_FilteredMeanIntensity.csv')
#df_filter_mi[df_filter_mi.index.str.contains('JE-TMA')].to_csv('20201210_JE-TMA-41-43-62_FilteredMeanIntensity.csv')

# Load Filtered Data <a name="load"></a>

Cells and markers filtered above

[contents](#contents)

In [ ]:
os.chdir(f'{filterdir}')
df_mi = pd.read_csv('20201210_JE-TMA-41-43-62_FilteredMeanIntensity.csv',index_col=0)
df_mi['batch'] = [item.split('_')[0] for item in df_mi.index]
df_mi['scene'] = [item.split('_')[1] for item in df_mi.index]
df_mi.drop('cell',axis=1,inplace=True)

## Data Sampling <a name="run"></a>

Sample 2400 cells per TMA for kBET evaluation. Also calculate the bg quantile normalization.

[contents](#contents)

In [ ]:
ls_sample = [#'SMT101Bx4-3','SMT101Bx2-3','SMT101Bx2-5',
 'JE-TMA-41', 'JE-TMA-43', 'JE-TMA-62']
ls_scene = ['scene01','scene07','scene04','scene03','scene05','scene08','scene11','scene12'] 

In [ ]:
os.chdir(datadir)

In [ ]:
d_bg = {}
df_result = pd.DataFrame(index=[item.split('_')[0] for item in ls_marker],columns=ls_sample)
for s_sample in ls_sample:
    df_bg = pd.read_csv(f'features_{s_sample}_BackgroundQuantiles.csv',index_col=0)
    se_bg = df_bg.loc[:,df_bg.columns.str.contains('_3')].mean()
    se_bg.index = [item.split('_')[0] for item in se_bg.index]
    for s_index in se_bg.index:
        if df_result.index.isin([s_index]).sum()!=0:
            df_result.loc[s_index,s_sample] = se_bg[s_index]
    d_bg.update({s_sample:se_bg})

In [ ]:
#save
if not os.path.exists('20201211_JE-TMA-41-43-62_bg-quant_normfactor.csv'):
    df_result.dropna().to_csv('20201211_JE-TMA-41-43-62_bg-quant_normfactor.csv')

In [ ]:
os.chdir(filterdir)

In [ ]:
#select 1000 rand cells for each scene in ls_scene
i_rand = 7
df_sample = pd.DataFrame()
for s_scene in ls_scene:
    #print(s_scene)
    df_scene = df_mi[df_mi.scene==s_scene]
    for s_sample in ls_sample:
        df_slide = df_scene[df_scene.batch==s_sample]
        df_sample = df_sample.append(df_slide.sample(n=300,  replace=False,  random_state=i_rand)) #7, 8, 9 (2)

In [ ]:
#save samples, bg quantile corrected
df_bg_corrected = pd.DataFrame(index=df_sample.index)
for s_sample in ls_sample:
    df_bg = pd.read_csv(f'{datadir}/features_{s_sample}_BackgroundQuantiles.csv',index_col=0)
    se_bg = df_bg.loc[:,df_bg.columns.str.contains('_3')].mean()
    se_bg.index = [item.split('_')[0] for item in se_bg.index]
    ls_index = df_sample[df_sample.batch==s_sample].index
    for s_marker in df_sample.columns[df_sample.dtypes=='float64']:
        if s_marker == 'DAPI2_nuclei':
            continue
        else:
            df_bg_corrected.loc[ls_index,s_marker] =  df_sample.loc[ls_index,s_marker]/se_bg.loc[s_marker.split('_')[0]]
df_bg_corrected['batch'] = [item.split('_')[0] for item in df_bg_corrected.index]
df_bg_corrected.to_csv(f'2020120{i_rand}_JE-TMA-41-43-62_SampledMeanIntensity_raw_bg-quant.csv')

In [ ]:
#plot histograms
%matplotlib inline
s_trans = 'raw'
bins=50
for s_marker in df_bg_corrected.columns[df_bg_corrected.dtypes=='float64']:
    print(s_marker)
    fig,ax=plt.subplots(2,1,figsize = (5,5))
    for idxs, s_batch in enumerate(sorted(set(df_sample.batch))):
        df_batch = df_sample[(df_sample.batch==s_batch)].loc[:,s_marker] + 1 #set minimum to 1
        if len(df_batch.dropna()) == 0:
            continue
        ax[0].hist(df_batch,bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
        ax[1].hist(df_bg_corrected.loc[df_bg_corrected.batch==s_batch,s_marker],bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
        ax[0].set_yscale('log')
        ax[1].set_yscale('log')
        ax[0].set_title(f'{s_marker.split("_")[0]}: raw Unscaled Data')
        ax[1].set_title(f'{s_marker.split("_")[0]}: Subcellular BG')
        ax[0].legend()
    plt.tight_layout()
    fig.savefig(f'{rootdir}/20201210/Different_Scaling_bg-quantile_{s_marker}_{s_trans}_{i_rand}.png')


In [ ]:
#save smpled data with transformations
np.log2(df_sample.loc[:,df_sample.dtypes=='float64']+1).merge(df_sample.loc[:,['batch']],left_index=True,right_index=True).to_csv(f'2020120{i_rand}_JE-TMA-41-43-62_SampledMeanIntensity_log2.csv')
np.arcsinh(df_sample.loc[:,df_sample.dtypes=='float64']).merge(df_sample.loc[:,['batch']],left_index=True,right_index=True).to_csv(f'2020120{i_rand}_JE-TMA-41-43-62_SampledMeanIntensity_arcsinh.csv')
(df_sample.loc[:,df_sample.dtypes=='float64']).merge(df_sample.loc[:,['batch']],left_index=True,right_index=True).to_csv(f'2020120{i_rand}_JE-TMA-41-43-62_SampledMeanIntensity_raw.csv')

## Scaling Function <a name="func"></a>

[contents](#contents)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
#functions
def test_scaling(X):
    distributions = [
     ('Unscaled data', X),
     ('Data after standard scaling',
        StandardScaler().fit_transform(X)),
     ('Data after min-max scaling',
        MinMaxScaler().fit_transform(X)),
     ('Data after max-abs scaling',
        MaxAbsScaler().fit_transform(X)),
     ('Data after robust scaling',
        RobustScaler(quantile_range=(25, 75),with_scaling=True).fit_transform(X)),
     ('Data after power transformation (Box-Cox)',
      PowerTransformer(method='box-cox').fit_transform(X)),
     ('Data after quantile transformation (gaussian pdf)',
        QuantileTransformer(output_distribution='normal')
        .fit_transform(X)),
      ]
    return(distributions)

# Different Scaling <a name="scale"></a>

Test scaling methods: standard, robust, min-max, power, etc.

[contents](#contents)

In [ ]:
#markers to norm
ls_marker= ['HER2_cellmem25', 'Vim_nucadj2', 'CD20_perinuc5', 'CD3_perinuc5',
       'CD31_perinuc5', 'CD4_perinuc5', 'CD44_perinuc5', 'CD45_perinuc5',
       'CD68_perinuc5', 'CD8_perinuc5', 'PD1_perinuc5', 'PDPN_perinuc5',
       'aSMA_perinuc5', 'CK14_cytoplasm', 'CK17_cytoplasm', 'CK19_cytoplasm',
       'CK5_cytoplasm', 'CK7_cytoplasm', 'Ecad_cytoplasm', 'DAPI2_nuclei',
       'ER_nuclei', 'Ki67_nuclei', 'LamAC_nuclei', 'PCNA_nuclei',
       'pHH3_nuclei']

In [ ]:
pwd

In [ ]:
#plot all cores, different scaling
%matplotlib inline
s_trans = 'raw'#'arcsinh'#'log2'#
bins=250
for s_marker in ls_marker:
    print(s_marker)
    fig,ax=plt.subplots(5,1,figsize = (5,10))
    for idxs, s_batch in enumerate(sorted(set(df_mi.batch))):
        df_batch = df_mi[(df_mi.batch==s_batch)].loc[:,s_marker] + 1 #set minimum to 1
        if len(df_batch.dropna()) == 0:
            continue
        if s_trans == 'log2':
            X = np.log2(df_batch.values.reshape(-1,1)) + 1
        elif s_trans == 'arcsinh':
            X = np.arcsinh(df_batch.values.reshape(-1,1))
        else:
            X = df_batch.values.reshape(-1,1)
        #print(X.min())
        distributions = test_scaling(X)
        for idx, tu_dist in enumerate(distributions):
            ax[idx].hist(tu_dist[1],bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
            ax[idx].set_title(f'{s_trans} {tu_dist[0]}')
            if s_trans == 'raw':
                if idx == 5:
                    pass
                elif idx == 6:
                    pass
                else:
                    ax[idx].set_yscale('log')
            ax[idx].set_xlim(np.quantile(tu_dist[1],.001),np.quantile(tu_dist[1],.999))
        ax[0].set_title(f'{s_marker.split("_")[0]}: {s_trans} Unscaled Data')
        ax[0].legend()
    plt.tight_layout()
    fig.savefig(f'{rootdir}/{s_date}/Different_Scaling_all_{s_marker}_{s_trans}.png')
    break

In [ ]:
#load bg-quant and restore
df_bgquant = pd.read_csv(f'{datadir}/20201211_JE-TMA-41-43-62_bg-quant_normfactor.csv',index_col=0)

In [ ]:
# scaling compare standard, robust, restore and bg quantile
%matplotlib inline
s_trans = 'raw'#'arcsinh'#'log2'#
bins=250
for s_marker in ls_marker:
    print(s_marker)
    fig,ax=plt.subplots(7,1,figsize = (5,12))
    for idxs, s_batch in enumerate(sorted(set(df_mi.batch))):
        df_batch = df_mi[(df_mi.batch==s_batch)].loc[:,s_marker] + 1 #set minimum to 1
        if len(df_batch.dropna()) == 0:
            continue
        if s_trans == 'log2':
            X = np.log2(df_batch.values.reshape(-1,1)) + 1
        else:
            X = df_batch.values.reshape(-1,1)
        dist_std = StandardScaler().fit_transform(X),
        dist_robust = RobustScaler(quantile_range=(25, 75),with_scaling=True).fit_transform(X),
        dist_restore = X/df_restore.loc[s_marker.split('_')[0],s_batch]
        dist_bgquant = X/df_bgquant.loc[s_marker.split('_')[0],s_batch]
        for idx, tu_dist in enumerate(distributions):
            ax[idx].hist(tu_dist[1],bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
            ax[idx].set_title(f'{s_trans} {tu_dist[0]}')
            if s_trans == 'raw':
                ax[idx].set_yscale('log')
            ax[idx].set_xlim(np.quantile(tu_dist[1],.001),np.quantile(tu_dist[1],.999))
        ax[0].set_title(f'{s_marker.split("_")[0]}: {s_trans} Unscaled Data')
        ax[0].legend()
    plt.tight_layout()
    fig.savefig(f'{rootdir}/{s_date}/Different_Scaling_all_{s_marker}_{s_trans}.png')
    break

In [ ]:
#load sampled data,scale, save for kBET analysis (no bug)
ls_sample = ['JE-TMA-41', 'JE-TMA-43', 'JE-TMA-62']
i_rand = 9
for s_trans in ['raw','log2','arcsinh']:
    df_sample = pd.read_csv(f'{filterdir}/2020120{i_rand}_JE-TMA-41-43-62_SampledMeanIntensity_{s_trans}.csv',index_col=0)
    #scaling results of sampled data
    dd_result = {}
    for s_batch in sorted(set(df_sample.batch)):
        for s_marker in ls_marker:
            d_result = {}
            df_batch = df_sample.loc[df_sample.batch==s_batch,s_marker] + 1
            X = df_batch.values.reshape(-1,1)
            distributions = test_scaling(X)
            for idx, tu_dist in enumerate(distributions[1:]):
                d_result.update({tu_dist[0].split('after ')[1].split(' ')[0]:tu_dist[1]})
            dd_result.update({f'{s_marker}_{s_batch}':d_result})
    #convert to dataframe
    df_result = pd.DataFrame()
    for s_marker_batch, d_result in dd_result.items():
        for s_norm, dist in d_result.items():
            df_result[f'{s_marker_batch}_{s_norm}'] = dist.reshape(1,dist.shape[0])[0]
    #save
    ls_norm = sorted(set([item.split('_')[-1] for item in df_result.columns]))
    for s_norm in ls_norm:
        df_norm = df_result.loc[:,df_result.columns.str.contains(s_norm)]
        df_out = pd.DataFrame()
        for s_sample in ls_sample:
            df_sample_norm = df_norm.loc[:,df_norm.columns.str.contains(s_sample)]
            df_sample_norm.columns = [item.split('_')[0] for item in df_sample_norm.columns]
            df_sample_norm['batch'] = s_sample
            df_out = df_out.append(df_sample_norm)
        df_out.index = df_sample.index
        df_out.to_csv(f'{filterdir}/2020120{i_rand}_JE-TMA-41-43-62_SampledMeanIntensity_{s_trans}_{s_norm}.csv')

In [ ]:
#list files generated
df_file = pd.DataFrame(index=os.listdir(filterdir))
#df_file[df_file.index.str.contains('20201209_JE-TMA-41-43-62_SampledMeanIntensity')].index.tolist()
df_file[df_file.index.str.contains('restore')].index.tolist()

## Scaling Visualization  <a name="norm"></a>

The standard scaling method preformed the best in kbet analysis. Visualize histograms of results.

[contents](#contents)

In [ ]:
#visualize

#plot histograms
#save
df['batch'] = [item.split('_')[0] for item in df.index]
#plot 
%matplotlib inline
s_trans = ''
s_date = "all"
bins=50
for s_marker in df.columns[df.dtypes=='float64']:
    print(s_marker)
    fig,ax=plt.subplots(2,1,figsize = (5,5))
    for idxs, s_batch in enumerate(sorted(set(df.batch))):
        df_batch = df[(df.batch==s_batch)].loc[:,s_marker] 
        if len(df_batch.dropna()) == 0:
            continue
        X = df_batch.values.reshape(-1,1)
        dist_std = StandardScaler().fit_transform(X),
        ax[0].hist(df.loc[df.index.str.contains(s_batch),s_marker],bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
        ax[1].hist(dist_std[0],bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
        ax[0].set_yscale('log')
        ax[1].set_yscale('log')
        ax[0].set_title(f'{s_marker.split("_")[0]}: Raw Data')
        ax[1].set_title(f'{s_marker.split("_")[0]}: Standard Scaling')
        ax[0].legend()
    plt.tight_layout()
    fig.savefig(f'{rootdir}/20201228/Different_Scaling_standard_{s_marker}_{s_trans}_{s_date}.png')

## kBET Results <a name="kbet"></a>

[contents](#contents)

In [ ]:
os.chdir(filterdir)
df_file = pd.DataFrame(index=os.listdir())
df_file = df_file[df_file.index.str.contains('JE-TMA-41-43-62_kbet_')]

In [ ]:
#add mean kbet
for s_file in df_file.index:
    df = pd.read_csv(s_file,index_col=0)
    df_file.loc[s_file,'mean_kbet'] = df.loc['mean','kBET.observed']
df_file['norm'] = [item.split('kbet_')[1].split('.csv')[0] for item in df_file.index]
df_file = df_file[df_file.norm!='raw_restore']

In [ ]:
ls_index= df_file.groupby('norm').mean_kbet.mean().sort_values().index
df_file.groupby('norm').mean_kbet.mean().sort_values()

In [ ]:
df_file['Norm'] = pd.Categorical(
    df_file['norm'], 
    categories=ls_index.tolist(), 
    ordered=True
)

In [ ]:
df_select = df_file[df_file.norm.isin(['raw_combat','raw_standard','restore_scale','raw','restore_div','restore_local','raw_regress_out'])] #'raw_restore_combat'

In [ ]:
%matplotlib inline
sns.set_style("whitegrid")
fig, ax = plt.subplots(figsize=(7,4))
sns.lineplot(data=df_file.sort_values('Norm'),x='norm',y='mean_kbet',ax=ax,err_style='bars')
labels = [item.replace('_out','').replace('div','global') for item in ls_index.tolist()]
ax.set_xticks(range(len(df_file.groupby('norm').mean_kbet)))
ax.set_xticklabels(labels,rotation=90)
ax.set_ylabel('Rejection Rate')
ax.set_xlabel('Normalization')
ax.set_title('kBET Evaluation of Batch Correction I')
fig.set_tight_layout(True)
plt.tight_layout
fig.savefig(f'{rootdir}/{s_date}/BatchEffectI.png',dpi=200)

In [ ]:
%matplotlib inline
sns.set_style("white")
fig, ax = plt.subplots(figsize=(4.2,3.2))
sns.lineplot(data=df_select.sort_values('mean_kbet'),x='norm',y='mean_kbet',ax=ax,err_style='bars')
labels = [item.replace('div','global').replace('raw_','').replace('_','\n').replace('raw','none') for item in df_select.groupby('norm').mean_kbet.mean().sort_values().index.tolist()]
ax.set_xticks(range(len(df_select.groupby('norm').mean_kbet)))
ax.set_xticklabels(labels,rotation=90)
ax.set_ylabel('Rejection Rate')
ax.set_xlabel('Normalization')
ax.set_title('kBET Evaluation of Batch Correction')
fig.set_tight_layout(True)
plt.tight_layout
fig.savefig(f'{rootdir}/{s_date}/BatchEffect_select.png',dpi=200)

In [ ]:
df_select.sort_index()

# Test Batch Correction <a name="viznorm"></a>

Apply combat and regress out algorithims

[contents](#contents)

In [ ]:
os.chdir(filterdir)
df_file = pd.DataFrame(index=os.listdir())
df_file = df_file[df_file.index.str.contains('20201207_JE-TMA-41-43-62_SampledMeanIntensity')]
#df_file.index.tolist()

In [ ]:
ls_index = [
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_robust.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_robust.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_combat.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_regress_out.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_power.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_min-max.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_max-abs.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_bg-quant.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_quantile.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2.csv',
 #'20201208_JE-TMA-41-43-62_SampledMeanIntensity_raw_restore.csv',
 #'20201209_JE-TMA-41-43-62_SampledMeanIntensity_raw_restore.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_quantile.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_max-abs.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_standard.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_power.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_min-max.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2_combat.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_regress_out.csv',
 #
 #top candidates
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_restore.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_combat.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw_standard.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_restore_scale.csv',
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_restore_local.csv'
 #'20201207_JE-TMA-41-43-62_SampledMeanIntensity_restore_div.csv'
 '20201207_JE-TMA-41-43-62_SampledMeanIntensity_restore_div_arcsinh.csv'
 ]
#ls_index = [ '20201208_JE-TMA-41-43-62_SampledMeanIntensity_raw.csv', '20201208_JE-TMA-41-43-62_SampledMeanIntensity_log2.csv',
#            '20201207_JE-TMA-41-43-62_SampledMeanIntensity_raw.csv', '20201207_JE-TMA-41-43-62_SampledMeanIntensity_log2.csv',
#            '20201209_JE-TMA-41-43-62_SampledMeanIntensity_raw.csv', '20201209_JE-TMA-41-43-62_SampledMeanIntensity_log2.csv']

In [ ]:
marker_genes = [ 'CD45_perinuc5',  'CD20_perinuc5','PD1_perinuc5', 'CD3_perinuc5', 'CD4_perinuc5',
 'CD8_perinuc5','CD68_perinuc5', 'aSMA_perinuc5','CD31_perinuc5', 'PDPN_perinuc5', 'Vim_nucadj2',
 'CD44_perinuc5', 'CK14_cytoplasm', 'CK5_cytoplasm','CK17_cytoplasm',  'CK19_cytoplasm', 'CK7_cytoplasm', 'Ecad_cytoplasm',
 'ER_nuclei','HER2_cellmem25', 'Ki67_nuclei', 'PCNA_nuclei', 'pHH3_nuclei', 'LamAC_nuclei']
marker_genes = ['CD45', 'CD20', 'PD1', 'CD3', 'CD4', 'CD8', 'CD68', 'aSMA', 'CD31', 'PDPN', 'Vim', 'CD44',
                'CK14','CK5', 'CK17', 'CK19', 'CK7', 'Ecad', 'ER', 'HER2', 'Ki67', 'PCNA', 'pHH3', 'LamAC']


In [ ]:
# normalize 
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
%matplotlib inline
sc.set_figure_params(scanpy=True, fontsize=14)
for s_index in ls_index:
    for s_norm in ['']: #,'none'#'regress_out','',,'combat'
        print(s_index)
        df = pd.read_csv(s_index.replace('kbet','SampledMeanIntensity'),index_col=0)
        df.columns = [item.split('_')[0] for item in df.columns]
        marker_genes = df.columns[df.dtypes=='float64'].tolist()
        adata = sc.AnnData(df.loc[:,df.dtypes=='float64'])
        adata.obs['batch'] = [item.split('_scene')[0] for item in adata.obs.index]
        adata.obs['scene'] = [item.split('_')[1] for item in adata.obs.index]
        #log transform, batch correct reduce dimensionality (PCA)
        #if s_type=='Raw':
        #    sc.pp.log1p(adata)
        adata.raw = adata
        #remove batch effect (non work well):#
        print(s_norm)
        if s_norm == 'combat':
            sc.pp.combat(adata,key='batch')
        elif s_norm == 'regress_out':
            sc.pp.regress_out(adata, keys='batch')
        elif s_norm == 'MNN':
            sc.external.pp.mnn_correct(adata,batch_key='batch') #didn't work
        else:
            print('')
        #reduce dimensionality
        sc.tl.pca(adata, svd_solver='auto')
        #save normalized data
        df = pd.DataFrame(data=adata.X,index=adata.obs.index,columns=adata.var.index)
        df['batch'] = [item.split('_scene')[0] for item in df.index]
        df.to_csv(f'{s_index.replace(".csv",f"_{s_norm}.csv")}')
        # calculate neighbors     
        sc.pp.neighbors(adata, n_neighbors=10, n_pcs=23)
        sc.tl.umap(adata)
        #umap plot
        s_type = s_index.split('SampledMeanIntensity_')[1].split('.')[0]
        figname = f"UmapBatch_{s_type}_{s_norm}.png"
        fig,ax = plt.subplots(figsize=(6,4))
        sc.pl.umap(adata, color='batch',title=f"{s_type.replace('_',' ')} {s_norm}",wspace=.25,ax=ax,save=figname)
        fig,ax = plt.subplots(figsize=(6,4))
        figname = f'UmapScene_{s_type}_{s_norm}.png'
        fig = sc.pl.umap(adata, color='scene',save=figname,ax=ax,title=f"{s_type.replace('_',' ')} {s_norm}")
        X_pca = adata.obsm['X_pca'] 
        # kmeans 
        '''
        k=12
        kmeans = KMeans(n_clusters=k, random_state=0).fit(X_pca) 
        adata.obs[f'kmeans{k}'] = kmeans.labels_.astype(str)
        figname=f'Umap_Kmeans_{s_type}_{s_norm}.png'
        fig,ax = plt.subplots(figsize=(6,4))
        sc.pl.umap(adata, color=f'kmeans{k}',save=figname,ax=ax)
        figname=f'Matrixplot_kmeans_{s_type}_{s_norm}.png'
        sc.pl.matrixplot(adata, var_names=marker_genes, groupby=f'kmeans{k}',log=True,dendrogram=True,save=figname)
        '''
        #leiden
        sc.tl.leiden(adata,resolution=0.25)
        fig,ax = plt.subplots(figsize=(6,4))
        figname=f'leiden_{s_type}_{s_norm}.png'
        sc.pl.umap(adata, color='leiden',ax=ax,save=figname)
        fig,ax = plt.subplots(figsize=(8,4))
        figname=f'Matrixplot_leiden_{s_type}_{s_norm}.png'
        sc.pl.matrixplot(adata, var_names=marker_genes, groupby=f'leiden',
                         dendrogram=True,ax=ax,save=figname,standard_scale='var',colorbar_title='Relative\nintensity')
        df = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
        df['leiden'] = adata.obs['leiden']
        break
    break
sc.pl.pca_variance_ratio(adata, log=True)

# Cluster composition  <a name="cluster"></a>

[contents](#contents)

In [ ]:
d_je_tma = {'scene01':'tonsil1',
    'scene02':'HCC1143', 
    'scene03':'HCC3153', 
    'scene04':'NBreast',
    'scene05':'T47D',
    'scene06':'T47D',
    'scene07':'tonsil2',
    'scene08':'BT474',
    'scene09':'BT474',
    'scene10':'AU565',
    'scene11':'AU565',
    'scene12':'MDAMB-436',
    'scene13':'MDAMB-436',
 }

In [ ]:
#stacked bar
s_trans = s_index.split('Intensity_')[1].split('.')[0]
df['slide'] = [item.split('_')[0] for item in df.index]
df['scene'] = [d_je_tma[item.split('_')[1]] for item in df.index]
df['slide_scene'] = df.slide + '_' + df.scene
df_prop = (df.groupby([f'leiden','slide_scene']).CD4.count())/(df.groupby(['slide_scene']).CD4.count())
df_prop = df_prop.unstack().fillna(value=0).T

fig,ax=plt.subplots(figsize=(7,3.7), dpi=200)
df_prop.columns = df_prop.columns.add_categories(['slide','scene'])
df_prop.index = [item.replace('JE-TMA-','') for item in df_prop.index]
df_prop['slide'] =[item.split('_')[0] for item in df_prop.index]
df_prop['scene'] =[item.split('_')[1] for item in df_prop.index]
df_prop.sort_values(['scene','slide']).plot(kind='bar',stacked=True,ax=ax,legend=True,cmap='tab20')
ax.legend(bbox_to_anchor=(1.02, 1.2), ncol=2)
ax.set_ylabel('Fraction Positive')
ax.set_title(f"{s_trans.replace('_',' ')} {s_norm}")
ax.grid(False)
plt.tight_layout()
fig.savefig(f'./figures/StackedBar_{s_trans}_{s_norm}_Leiden.png')


# implement combat for fit/transform <a name="combat"></a>

[contents](#contents)

In [ ]:
os.chdir(f'{filterdir}')
df = pd.read_csv('20201210_JE-TMA-41-43-62_FilteredMeanIntensity.csv',index_col=0)
df['batch'] = [item.split('_')[0] for item in df.index]
df['scene'] = [item.split('_')[1] for item in df.index]
df.drop('cell',axis=1,inplace=True)

In [ ]:
#load raw data
#df = pd.read_csv(f'{s_index.replace(".csv",f"_{s_norm}.csv")}',index_col=0)
#visualize raw
adata = sc.AnnData(df.loc[df.index.str.contains('JE-TMA'),df.dtypes=='float64'])
adata.obs['batch'] = [item.split('_scene')[0] for item in adata.obs.index]
adata.obs['scene'] = [item.split('_')[1] for item in adata.obs.index]
sc.tl.pca(adata, svd_solver='auto')
# calculate neighbors     
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=23)
sc.tl.umap(adata)
#umap plot
sc.pl.umap(adata, color='batch')

In [ ]:
%matplotlib inline
#umap plot
s_type = 'Full_TMA'
s_norm = 'raw'
figname = f"UmapBatch_{s_type}_{s_norm}.png"
fig,ax = plt.subplots(figsize=(6,4))
sc.pl.umap(adata, color='batch',title=f"{s_type.replace('_',' ')} {s_norm}",wspace=.25,ax=ax,save=figname)

In [ ]:
figname = f"UmapScene_{s_type}_{s_norm}.png"
fig,ax = plt.subplots(figsize=(6,4))
sc.pl.umap(adata, color='scene',title=f"{s_type.replace('_',' ')} {s_norm}",wspace=.25,ax=ax,save=figname)

In [ ]:
#from https://github.com/brentp/combat.py/blob/master/combat.py
import patsy
import sys
import numpy.linalg as la

def aprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = gamma_hat.var()
    return (2 * s2 +m**2) / s2

def bprior(gamma_hat):
    m = gamma_hat.mean()
    s2 = gamma_hat.var()
    return (m*s2+m**3)/s2

def it_sol(sdat, g_hat, d_hat, g_bar, t2, a, b, conv=0.0001):
    n = (1 - np.isnan(sdat)).sum(axis=1)
    g_old = g_hat.copy()
    d_old = d_hat.copy()

    change = 1
    count = 0
    while change > conv:
        #print g_hat.shape, g_bar.shape, t2.shape
        g_new = postmean(g_hat, g_bar, n, d_old, t2)
        sum2 = ((sdat - np.dot(g_new.values.reshape((g_new.shape[0], 1)), np.ones((1, sdat.shape[1])))) ** 2).sum(axis=1)
        d_new = postvar(sum2, n, a, b)
       
        change = max((abs(g_new - g_old) / g_old).max(), (abs(d_new - d_old) / d_old).max())
        g_old = g_new #.copy()
        d_old = d_new #.copy()
        count = count + 1
    adjust = (g_new, d_new)
    return adjust 

def postmean(g_hat, g_bar, n, d_star, t2):
    return (t2*n*g_hat+d_star * g_bar) / (t2*n+d_star)

def postvar(sum2, n, a, b):
    return (0.5 * sum2 + b) / (n / 2.0 + a - 1.0)

def design_mat(mod, numerical_covariates, batch_levels):
    # require levels to make sure they are in the same order as we use in the
    # rest of the script.
    design = patsy.dmatrix("~ 0 + C(batch, levels=%s)" % str(batch_levels),
                                                  mod, return_type="dataframe")

    mod = mod.drop(["batch"], axis=1)
    numerical_covariates = list(numerical_covariates)
    sys.stderr.write("found %i batches\n" % design.shape[1])
    other_cols = [c for i, c in enumerate(mod.columns)
                  if not i in numerical_covariates]
    factor_matrix = mod[other_cols]
    design = pd.concat((design, factor_matrix), axis=1)
    if numerical_covariates is not None:
        sys.stderr.write("found %i numerical covariates...\n"
                            % len(numerical_covariates))
        for i, nC in enumerate(numerical_covariates):
            cname = mod.columns[nC]
            sys.stderr.write("\t{0}\n".format(cname))
            design[cname] = mod[mod.columns[nC]]
    sys.stderr.write("found %i categorical variables:" % len(other_cols))
    sys.stderr.write("\t" + ", ".join(other_cols) + '\n')
    return design

def combat(data, batch, model=None, numerical_covariates=None):
    """Correct for batch effects in a dataset
    Parameters
    ----------
    data : pandas.DataFrame
        A (n_features, n_samples) dataframe of the expression or methylation
        data to batch correct
    batch : pandas.Series
        A column corresponding to the batches in the data, with index same as
        the columns that appear in ``data``
    model : patsy.design_info.DesignMatrix, optional
        A model matrix describing metadata on the samples which could be
        causing batch effects. If not provided, then will attempt to coarsely
        correct just from the information provided in ``batch``
    numerical_covariates : list-like
        List of covariates in the model which are numerical, rather than
        categorical
    Returns
    -------
    corrected : pandas.DataFrame
        A (n_features, n_samples) dataframe of the batch-corrected data
    """
    if isinstance(numerical_covariates, str):
        numerical_covariates = [numerical_covariates]
    if numerical_covariates is None:
        numerical_covariates = []

    if model is not None and isinstance(model, pd.DataFrame):
        model["batch"] = list(batch)
    else:
        model = pd.DataFrame({'batch': batch})

    batch_items = model.groupby("batch").groups.items()
    batch_levels = [k for k, v in batch_items]
    batch_info = [v for k, v in batch_items]
    n_batch = len(batch_info)
    n_batches = np.array([len(v) for v in batch_info])
    n_array = float(sum(n_batches))

    # drop intercept
    drop_cols = [cname for cname, inter in  ((model == 1).all()).iteritems() if inter == True]
    drop_idxs = [list(model.columns).index(cdrop) for cdrop in drop_cols]
    model = model[[c for c in model.columns if not c in drop_cols]]
    numerical_covariates = [list(model.columns).index(c) if isinstance(c, str) else c
            for c in numerical_covariates if not c in drop_cols]

    design = design_mat(model, numerical_covariates, batch_levels)

    sys.stderr.write("Standardizing Data across genes.\n")
    #error shapes (3,7200) and (26,7200) not aligned: 7200 (dim 1) != 26 (dim 0)
    B_hat = np.dot(np.dot(la.inv(np.dot(design.T, design)), design.T), data.T) #data.T
    grand_mean = np.dot((n_batches / n_array).T, B_hat[:n_batch,:])
    var_pooled = np.dot(((data - np.dot(design, B_hat).T)**2), np.ones((int(n_array), 1)) / int(n_array))

    stand_mean = np.dot(grand_mean.T.reshape((len(grand_mean), 1)), np.ones((1, int(n_array))))
    tmp = np.array(design.copy())
    tmp[:,:n_batch] = 0
    stand_mean  += np.dot(tmp, B_hat).T

    s_data = ((data - stand_mean) / np.dot(np.sqrt(var_pooled), np.ones((1, int(n_array)))))

    sys.stderr.write("Fitting L/S model and finding priors\n")
    batch_design = design[design.columns[:n_batch]]
    gamma_hat = np.dot(np.dot(la.inv(np.dot(batch_design.T, batch_design)), batch_design.T), s_data.T)

    delta_hat = []

    for i, batch_idxs in enumerate(batch_info):
        #batches = [list(model.columns).index(b) for b in batches]
        delta_hat.append(s_data[batch_idxs].var(axis=1))

    gamma_bar = gamma_hat.mean(axis=1) 
    t2 = gamma_hat.var(axis=1)
   

    a_prior = list(map(aprior, delta_hat))
    b_prior = list(map(bprior, delta_hat))

    sys.stderr.write("Finding parametric adjustments\n")
    gamma_star, delta_star = [], []
    for i, batch_idxs in enumerate(batch_info):
        #print '18 20 22 28 29 31 32 33 35 40 46'
        #print batch_info[batch_id]

        temp = it_sol(s_data[batch_idxs], gamma_hat[i],
                     delta_hat[i], gamma_bar[i], t2[i], a_prior[i], b_prior[i])

        gamma_star.append(temp[0])
        delta_star.append(temp[1])

    sys.stdout.write("Adjusting data\n")
    bayesdata = s_data
    gamma_star = np.array(gamma_star)
    delta_star = np.array(delta_star)


    for j, batch_idxs in enumerate(batch_info):

        dsq = np.sqrt(delta_star[j,:])
        dsq = dsq.reshape((len(dsq), 1))
        denom =  np.dot(dsq, np.ones((1, n_batches[j])))
        numer = np.array(bayesdata[batch_idxs] - np.dot(batch_design.loc[batch_idxs], gamma_star).T)

        bayesdata[batch_idxs] = numer / denom
   
    vpsq = np.sqrt(var_pooled).reshape((len(var_pooled), 1))
    bayesdata = bayesdata * np.dot(vpsq, np.ones((1, int(n_array)))) + stand_mean
 
    return bayesdata


In [ ]:
#test function as is
data = df.loc[:,df.dtypes=='float64'].T
batch = df.batch
bayesdata = combat(data, batch)

In [ ]:
#adapted from https://github.com/brentp/combat.py/blob/master/combat.py
def combat_fit(data, batch, model=None, numerical_covariates=None):
    """Correct for batch effects in a dataset
    Parameters
    ----------
    data : pandas.DataFrame
        A (n_features, n_samples) dataframe of the expression or methylation
        data to batch correct
    batch : pandas.Series
        A column corresponding to the batches in the data, with index same as
        the columns that appear in ``data``
    model : patsy.design_info.DesignMatrix, optional
        A model matrix describing metadata on the samples which could be
        causing batch effects. If not provided, then will attempt to coarsely
        correct just from the information provided in ``batch``
    numerical_covariates : list-like
        List of covariates in the model which are numerical, rather than
        categorical
    Returns
    -------
    gamma_star : centering parameters from combat fitting
    delta_star : scaling parameters from combat fitting
    """
    if isinstance(numerical_covariates, str):
        numerical_covariates = [numerical_covariates]
    if numerical_covariates is None:
        numerical_covariates = []

    if model is not None and isinstance(model, pd.DataFrame):
        model["batch"] = list(batch)
    else:
        model = pd.DataFrame({'batch': batch})

    batch_items = model.groupby("batch").groups.items()
    batch_levels = [k for k, v in batch_items]
    batch_info = [v for k, v in batch_items]
    n_batch = len(batch_info)
    n_batches = np.array([len(v) for v in batch_info])
    n_array = float(sum(n_batches))

    # drop intercept
    drop_cols = [cname for cname, inter in  ((model == 1).all()).iteritems() if inter == True]
    drop_idxs = [list(model.columns).index(cdrop) for cdrop in drop_cols]
    model = model[[c for c in model.columns if not c in drop_cols]]
    numerical_covariates = [list(model.columns).index(c) if isinstance(c, str) else c
            for c in numerical_covariates if not c in drop_cols]

    design = design_mat(model, numerical_covariates, batch_levels)

    sys.stderr.write("Standardizing Data across genes.\n")
    B_hat = np.dot(np.dot(la.inv(np.dot(design.T, design)), design.T), data.T) 
    grand_mean = np.dot((n_batches / n_array).T, B_hat[:n_batch,:])
    var_pooled = np.dot(((data - np.dot(design, B_hat).T)**2), np.ones((int(n_array), 1)) / int(n_array))

    stand_mean = np.dot(grand_mean.T.reshape((len(grand_mean), 1)), np.ones((1, int(n_array))))
    tmp = np.array(design.copy())
    tmp[:,:n_batch] = 0
    stand_mean  += np.dot(tmp, B_hat).T

    s_data = ((data - stand_mean) / np.dot(np.sqrt(var_pooled), np.ones((1, int(n_array)))))

    sys.stderr.write("Fitting L/S model and finding priors\n")
    batch_design = design[design.columns[:n_batch]]
    gamma_hat = np.dot(np.dot(la.inv(np.dot(batch_design.T, batch_design)), batch_design.T), s_data.T)

    delta_hat = []

    for i, batch_idxs in enumerate(batch_info):
        delta_hat.append(s_data[batch_idxs].var(axis=1))

    gamma_bar = gamma_hat.mean(axis=1) 
    t2 = gamma_hat.var(axis=1)
   

    a_prior = list(map(aprior, delta_hat))
    b_prior = list(map(bprior, delta_hat))

    sys.stderr.write("Finding parametric adjustments\n")
    gamma_star, delta_star = [], []
    for i, batch_idxs in enumerate(batch_info):
        temp = it_sol(s_data[batch_idxs], gamma_hat[i],
                     delta_hat[i], gamma_bar[i], t2[i], a_prior[i], b_prior[i])

        gamma_star.append(temp[0])
        delta_star.append(temp[1])
    return(gamma_star, delta_star)
        
def combat_transform(data, batch, gamma_star, delta_star,model=None, numerical_covariates=None):
    """Correct for batch effects in a dataset
    Parameters
    ----------
    data : pandas.DataFrame
        A (n_features, n_samples) dataframe of the expression or methylation
        data to batch correct
    batch : pandas.Series
        A column corresponding to the batches in the data, with index same as
        the columns that appear in ``data``
    gamma_star : centering parameters from combat fitting
    delta_star : scaling parameters from combat fitting
    model : patsy.design_info.DesignMatrix, optional
        A model matrix describing metadata on the samples which could be
        causing batch effects. If not provided, then will attempt to coarsely
        correct just from the information provided in ``batch``
    numerical_covariates : list-like
        List of covariates in the model which are numerical, rather than
        categorical
    Returns
    -------
    corrected : pandas.DataFrame
        A (n_features, n_samples) dataframe of the batch-corrected data
    """
    #get design
    if isinstance(numerical_covariates, str):
        numerical_covariates = [numerical_covariates]
    if numerical_covariates is None:
        numerical_covariates = []

    if model is not None and isinstance(model, pd.DataFrame):
        model["batch"] = list(batch)
    else:
        model = pd.DataFrame({'batch': batch})
    batch_items = model.groupby("batch").groups.items()
    batch_levels = [k for k, v in batch_items]
    batch_info = [v for k, v in batch_items]
    n_batch = len(batch_info)
    n_batches = np.array([len(v) for v in batch_info])
    n_array = float(sum(n_batches))
    # drop intercept
    drop_cols = [cname for cname, inter in  ((model == 1).all()).iteritems() if inter == True]
    drop_idxs = [list(model.columns).index(cdrop) for cdrop in drop_cols]
    model = model[[c for c in model.columns if not c in drop_cols]]
    numerical_covariates = [list(model.columns).index(c) if isinstance(c, str) else c
            for c in numerical_covariates if not c in drop_cols]
    design = design_mat(model, numerical_covariates, batch_levels)
    #standardize
    sys.stderr.write("Standardizing Data across genes.\n")
    B_hat = np.dot(np.dot(la.inv(np.dot(design.T, design)), design.T), data.T) 
    grand_mean = np.dot((n_batches / n_array).T, B_hat[:n_batch,:])
    var_pooled = np.dot(((data - np.dot(design, B_hat).T)**2), np.ones((int(n_array), 1)) / int(n_array))

    stand_mean = np.dot(grand_mean.T.reshape((len(grand_mean), 1)), np.ones((1, int(n_array))))
    tmp = np.array(design.copy())
    tmp[:,:n_batch] = 0
    stand_mean  += np.dot(tmp, B_hat).T
    s_data = ((data - stand_mean) / np.dot(np.sqrt(var_pooled), np.ones((1, int(n_array)))))
    batch_design = design[design.columns[:n_batch]]
    # adjust data
    sys.stdout.write("Adjusting data\n")
    bayesdata = s_data
    gamma_star = np.array(gamma_star)
    delta_star = np.array(delta_star)
    #for each batch
    for j, batch_idxs in enumerate(batch_info):

        dsq = np.sqrt(delta_star[j,:])
        dsq = dsq.reshape((len(dsq), 1))
        denom =  np.dot(dsq, np.ones((1, n_batches[j]))) #divide by sqrt delta_star
        numer = np.array(bayesdata[batch_idxs] - np.dot(batch_design.loc[batch_idxs], gamma_star).T) #subtract gamma_star

        bayesdata[batch_idxs] = numer / denom
    #multiply by square root of variance and add mean
    vpsq = np.sqrt(var_pooled).reshape((len(var_pooled), 1))
    bayesdata = bayesdata * np.dot(vpsq, np.ones((1, int(n_array)))) + stand_mean
    return bayesdata


In [ ]:
#fit
gamma_star, delta_star = combat_fit(data, batch)

In [ ]:
gamma_star[0] #centering factor
delta_star[0][0]  #scaling factor

In [ ]:
#transform
bayesdata = combat_transform(data,batch,gamma_star, delta_star)

# Visualize combat histograms <a name="combatviz"></a>

[contents](#contents)

In [ ]:
df_norm = bayesdata.T

In [ ]:
#visualize

#plot histograms
#save
df_norm['batch'] = [item.split('_')[0] for item in df_norm.index]
#plot 
%matplotlib inline
s_trans = ''
s_date = "all"
bins=50
for s_marker in df_norm.columns[df_norm.dtypes=='float64']:
    print(s_marker)
    fig,ax=plt.subplots(2,1,figsize = (5,5))
    for idxs, s_batch in enumerate(sorted(set(df_norm.batch))):
        df_batch = df_norm[(df_norm.batch==s_batch)].loc[:,s_marker] #+ 1 #set minimum to 1
        if len(df_batch.dropna()) == 0:
            continue
        ax[1].hist(df_batch,bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
        ax[0].hist(df.loc[df.index.str.contains(s_batch),s_marker],bins=bins,alpha=0.4, color=f'C{idxs}',label=s_batch)
        ax[0].set_yscale('log')
        ax[1].set_yscale('log')
        ax[0].set_title(f'{s_marker.split("_")[0]}: Raw Data')
        ax[1].set_title(f'{s_marker.split("_")[0]}: Combat')
        ax[0].legend()
    plt.tight_layout()
    fig.savefig(f'{rootdir}/20201228/Different_Scaling_combat_{s_marker}_{s_trans}_{s_date}.png')

# Test on Biopsies

In [ ]:
#test on biopsy samples
df_test = pd.read_csv('20201228_SMT101-Bx2-Bx2-Bx4_FilteredMeanIntensity.csv',index_col=0)
df_test.drop(['slide_scene','cell'], axis=1,inplace=True)
df_test['batch'] = [item.split('_scene')[0] for item in df_test.index]
df_sample = pd.DataFrame()
df_control = pd.DataFrame() 
for s_batch in ['SMT101Bx2-3', 'SMT101Bx2-5', 'SMT101Bx4-3']:
    df_sample = df_sample.append(df_test[df_test.batch==s_batch])
for s_batch in ['JE-TMA-41', 'JE-TMA-43', 'JE-TMA-62']:
    df_control = df_control.append(df_test[df_test.batch==s_batch])

In [ ]:
#fit
df=df_control
data = df.loc[:,df.dtypes=='float64'].T
batch = df.batch
gamma_star, delta_star = combat_fit(data, batch)
#transform
df=df_sample
data = df.loc[:,df.dtypes=='float64'].T
batch = df.batch
bayesdata = combat_transform(data,batch,gamma_star, delta_star)
df_norm = bayesdata.T

In [ ]:
#transform control

df=df_control
data = df.loc[:,df.dtypes=='float64'].T
batch = df.batch
bayesdata = combat_transform(data,batch,gamma_star, delta_star)
df_norm_control = bayesdata.T


In [ ]:
#df_sample

In [ ]:
#raw
#load normalized data
df = df_sample.append(df_control)
#visualize
adata = sc.AnnData(df.loc[:,df.dtypes=='float64'])
adata.obs['batch'] = [item.split('_scene')[0] for item in adata.obs.index]
adata.obs['scene'] = [item.split('_')[1] for item in adata.obs.index]
sc.tl.pca(adata, svd_solver='auto')
# calculate neighbors     
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=23)
sc.tl.umap(adata)
#umap plot
sc.pl.umap(adata, color='batch')

In [ ]:
#umap plot
s_type = 'Biopsy & control'
s_norm = 'raw'
figname = f"UmapBatch_{s_type}_{s_norm}.png"
fig,ax = plt.subplots(figsize=(6,4))
sc.pl.umap(adata, color='batch',title=f"{s_type.replace('_',' ')} {s_norm}",wspace=.25,ax=ax,save=figname)

In [ ]:
#load normalized data
df = df_norm.append(df_norm_control)
#visualize
#adata = sc.AnnData(df.loc[:,ls_markers])
adata = sc.AnnData(df.loc[:,df.dtypes=='float64'])
adata.obs['batch'] = [item.split('_scene')[0] for item in adata.obs.index]
adata.obs['scene'] = [item.split('_')[1] for item in adata.obs.index]
sc.tl.pca(adata, svd_solver='auto')
# calculate neighbors     
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=19)
sc.tl.umap(adata)
#umap plot
sc.pl.umap(adata, color='batch')

In [ ]:
#umap plot
s_type = 'Biopsy & control'#'Control'
s_norm = 'combat'
figname = f"UmapBatch_{s_type}_{s_norm}.png"
fig,ax = plt.subplots(figsize=(6,4))
sc.pl.umap(adata, color='batch',title=f"{s_type.replace('_',' ')} {s_norm}",wspace=.25,ax=ax,save=figname)

In [ ]:
#cluster leiden
sc.tl.leiden(adata, resolution = 0.25)

In [ ]:
figname = f"UmapLeiden_{s_type}_{s_norm}.png"
sc.pl.umap(adata, color='leiden',wspace=.25,save=figname)

In [ ]:
adata_smt = adata[adata.obs['leiden'].isin({'5', '6'}),:]

In [ ]:
sc.tl.rank_genes_groups(adata_smt, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata_smt, n_genes=10, sharey=False)

In [ ]:
marker_genes = ['CD45_perinuc5','CD3_perinuc5','PDPN_perinuc5','CK5_cytoplasm',
                'CK17_cytoplasm','Ki67_nuclei','CD44_perinuc5','CD8_perinuc5',
               'CK19_cytoplasm','pHH3_nuclei','CD20_perinuc5','Vim_nucadj2']

In [ ]:
ls_markers = ['HER2_cellmem25', 'Vim_nucadj2',  'CD3_perinuc5',#
       'CD31_perinuc5', 'CD4_perinuc5', 'CD44_perinuc5', #'
       'CD68_perinuc5', 'CD8_perinuc5', 'PD1_perinuc5', 'PDPN_perinuc5',
       'aSMA_perinuc5', 'CK14_cytoplasm', 'CK17_cytoplasm', #
       'CK5_cytoplasm',  'Ecad_cytoplasm', 'DAPI2_nuclei',#,
       'ER_nuclei', 'Ki67_nuclei', 'LamAC_nuclei', 'PCNA_nuclei',
              'pHH3_nuclei','CD20_perinuc5','CD45_perinuc5','CK19_cytoplasm','CK7_cytoplasm']
#umap plot
s_type = 'Biopsy & control'#'Control'
s_norm = 'combat'
figname = f"UmapMarkers_{s_type}_{s_norm}.png"
sc.pl.umap(adata, color=marker_genes,wspace=.25,save=figname)

In [ ]:
#CD20,CK19,CD45,CK7

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
figname = f'UmapScene_{s_type}_{s_norm}.png'
fig = sc.pl.umap(adata, color='scene',save=figname,ax=ax,title=f"{s_type.replace('_',' ')} {s_norm}")